<a href="https://colab.research.google.com/github/Gunasheela-KS/NLP/blob/main/Document_QA_Langchain_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# !pip install --upgrade langchain openai -q
# !pip install unstructured -q
# !pip install unstructured[local-inference] -q
# !pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6
# !apt-get install poppler-utils
# !pip install tiktoken -q
# !pip install pinecone-client -q
# !pip install --upgrade PIL
import os
import openai
import pinecone
import PIL
from PIL import Image
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [48]:
os.environ['OPENAI_API_KEY']='your_key'

In [49]:
directory = '/content/data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)

2

In [50]:
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

4


In [51]:
embeddings = OpenAIEmbeddings(model="ada")

In [52]:
pinecone.init(
    api_key="your_pinecone_api_key",
    environment="your_env"
)
index_name = "langchain-demo"

In [53]:
pinecone.create_index(index_name, dimension=1536,
                          metric="cosine", pods=1, pod_type="p1.x1")

In [54]:
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [56]:
def get_similiar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs

def get_answer(query):
  similar_docs = get_similiar_docs(query)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer

In [58]:
# model_name = "text-davinci-003"
# model_name = "gpt-3.5-turbo"
model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name)

chain = load_qa_chain(llm, chain_type="stuff")

In [63]:
query = "What is pinecone?"
answer = get_answer(query)
print(answer)

Pinecone is an indexing and similarity search system developed by OpenAI that provides a vector database for storing and retrieving vectors based on their similarity, particularly well-suited for applications such as recommendation systems, natural language processing, and computer vision. It leverages advanced algorithms and data structures, such as approximate nearest neighbor (ANN) search, and provides a simple API for easy integration into applications. By using Pinecone, developers can create highly performant and accurate recommendation systems, personalized search engines, content matching systems, and more.
